In [6]:
from repairing_genomic_gaps import cae_200
from repairing_genomic_gaps.utils import get_model_history_path, get_model_weights_path
from repairing_genomic_gaps import build_multivariate_dataset_cae, build_synthetic_dataset_cae

In [5]:
window_size = 200
model = cae_200()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 200, 4)]          0         
_________________________________________________________________
reshape (Reshape)            (None, 200, 4, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 100, 4, 64)        2624      
_________________________________________________________________
batch_normalization (BatchNo (None, 100, 4, 64)        256       
_________________________________________________________________
activation (Activation)      (None, 100, 4, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 2, 32)         81952     
_________________________________________________________________
batch_normalization_1 (Batch (None, 20, 2, 32)         128 

In [ ]:
train, test = build_synthetic_dataset_cae(window_size, batch_size=512)